# Using the pipeline together with COBRApy functions to studying auxotrophy

In this tutorial the pipeline will suggest the reactions to knock-in to allow _E. coli_ auxotrophic for Tryptophan to grow on methane and produce Arginine. 

It is already known that *E. coli* can't grow on methane thus reactions should be added to give the model this functionality. So the approach is to:

1. make the model iML1515 auxotrophic for Tryptophan
2. add the Trp back to the medium to restore growth
3. on these conditions run the analysis for growth on methane and production of Arginine


## 1) Make the model auxotrophic for Trp

For this some previous research is needed:
- It is important to find out which genes are usually knocked-out to make a stain of *E. coli* auxotrophic for Trp
- Then the correspondant reaction(s) that is catalyzed by the enzyme encoded by the gene(s) have to be identified. For this  the [BiGG database](http://bigg.ucsd.edu/) is of help. Typing the gene in there allow you to find the reactions that are associated to it.
For instance in the following example, it was known that trpC is the gene that is commonly knocked-out to make *E. coli* auxotrophic for Trp, therefore trpc was typed in the BiGG search bar and the first lines of the results are the following

<img src="./images/aux1.png" alt="IGPS reaction search" title="Bigg" width="500" height="100" />

The model that is being used (i.e iML1515) is not included in the result, however, the reaction accociated to trpC in iML1515 can be found by following the link of the gene. In this case the first solution was clicked and the following is the information on gene reaction association:

<img src="./images/aux2.png" alt="IGPS reaction search" title="Bigg" width="500" height="100" />

Then clicking in one of the two reactions you could find on the right the list of the models in which it is found (see the red arro in the picture below).

<img src="./images/aux3.png" alt="IGPS reaction search" title="IGPS page & models" width="500" height="100" />

- Once the reaction(s) correspondant to the gene KO is identified, those should be removed from the model which should not grow in a normal medium composition



In [1]:
#inport statements

from os.path import join
import cobra

from pipeline.scripts.analysis import analysis_gf_sol, dict_prod_sol, cons_prod_dict, add_sol_cons
from pipeline.scripts.import_models import get_universal_main, get_reference_model
from pipeline.scripts.input_parser import main
from pipeline.scripts.scoring_system import (scores_evaluations, score_BB_presence, 
                            plotting_KI_BB_scores)
from pipeline.scripts.to_and_from_biobrick_wikidata import (output_for_biobrick_search, \
                                        run_harmonization_per_model)
from pipeline.scripts.path_definition_mdf import mdf_analysis
from pipeline.scripts.call_Optknock import (run_optknock_analysis, \
                                        add_Optknock_to_analysis)
from pipeline.scripts.output_generation import (summary_output, metab_out_chain, 
                            write_data_provenance)

Using license file C:\Users\delip\gurobi.lic
Academic license - for non-commercial use only
No parameters matching '_test' found


In [2]:
data_repo = "./pipeline/inputs"
model_aux = get_reference_model(data_repo, './pipeline/inputs/ecoli_ch4_arg.csv')
universal = get_universal_main(data_repo, './pipeline/inputs/ecoli_ch4_arg.csv')

In [3]:
trpC = model_aux.reactions.IGPS

In [4]:
trpC

Reaction identifier,IGPS
Name,Indole-3-glycerol-phosphate synthase
Memory address,0x01f3a50cba58
Stoichiometry,2cpr5p_c + h_c --> 3ig3p_c + co2_c + h2o_c 1-(2-Carboxyphenylamino)-1-deoxy-D-ribulose 5-phosphate + H+ --> C'-(3-Indolyl)-glycerol 3-phosphate + CO2 CO2 + H2O H2O
GPR,b1262
Lower bound,0.0
Upper bound,1000.0


In [5]:
growth_wt = model_aux.optimize()

In [6]:
growth_wt.objective_value

0.8769972144269707

In [7]:
growth_wt.fluxes['IGPS']

0.04985115265967229

In [8]:
for i in model_aux.reactions:
    if i.flux <= -0.5 and "EX_" in i.id:
        print(i.id, i.reaction, i.flux)

EX_pi_e pi_e <=>  -0.8459567750195305
EX_glc__D_e glc__D_e <=>  -10.0
EX_nh4_e nh4_e <=>  -9.471495371048078
EX_o2_e o2_e <=>  -22.131763238945883


In [9]:
for i in model_aux.reactions:
    if i.flux >= 0.5 and "EX_" in i.id:
        print(i.id, i.reaction, i.flux)

EX_co2_e co2_e <=>  24.003293272975927
EX_h_e h_e <=>  8.058200328043645
EX_h2o_e h2o_e <=>  47.16236480869445


In [10]:
model_aux.remove_reactions([trpC])

In [11]:
growth_ko = model_aux.optimize()

In [12]:
growth_ko.objective_value

0.0

###### Considerations
- Eliminating the reactions associated to trpC causes no growth on the wild type carbon source (glucose)
- Adding the amino acid, Trp in this case, to the medium should restore growth

## 2) Adding Trp to the medium to restore growth

In [13]:
trpgex = model_aux.reactions.EX_trp__L_e

In [14]:
trpgex.bounds

(0.0, 1000.0)

In [15]:
trpgex.lower_bound = -0.05 #inverse of the flux through the reacion KO in the wt model

In [16]:
growth_ms = model_aux.optimize()

In [17]:
growth_ms.objective_value

0.879615783825625

## 3) Using the function of the pipeline to find out which reactions should be added

*E. coli* can't grow on methane, and the strain auxotrophic for Trp can't grow on it either. Therefore GapFilling is needed to find out possible reaction addition.  

In [19]:
main('./pipeline/inputs/ecoli_ch4_arg.csv', universal, model_aux)

For ch4 there isn't any uptake trasnsporter in the reference model
The trasporter has been added to the 
                                reference model from the input file

For arg__L there is already a transport reaction allowing the uptake from the periplasmic space:  
Reaction ID:  ARGAGMt7pp 
Reaction equation:  agm_c + arg__L_p <=> agm_p + arg__L_c

The following reactions will be added to the universal model: ['R01142', 'R01143', 'MMO1', 'MMO2']
unknown metabolite 'ch4_c' created
unknown metabolite 'focytcc_c' created
unknown metabolite 'ficytcc_c' created


Name,iML1515
Memory address,0x0252050dba58
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [22]:
consumption = analysis_gf_sol('./pipeline/inputs/ecoli_ch4_arg.csv', model_aux, universal)

EX_ch4_e  |  ch4_e -->   |  (0.0, 1000.0)
EX_arg__L_e  |  arg__L_e -->   |  (0.0, 1000.0)
Old biomass (objective) bounds =  (0.0, 1000.0)
EX_pi_e pi_e <=>  -0.8459567750194139
EX_glc__D_e glc__D_e <=>  -10.0
EX_nh4_e nh4_e <=>  -9.471495371047927
EX_o2_e o2_e <=>  -22.131763238945897
New biomass (objective) bounds =  (0.0, 0.8769972144269785)

ch4 is in the medium 

Exchange ch4:  ch4_e -->  Old bounds:  (0.0, 1000.0)

arg__L is in the medium 

Exchange arg__L:  arg__L_e -->  Old bounds:  (0.0, 1000.0)
Starting reaction search with GapFilling . . .

---Model 1---

Reaction MMO2, solution of round 1 has been added to the model

Reaction ALCD1, solution of round 1 has been added to the model

Growth rate:  0.8769972144269785

The flux throughr EX_ch4_e is:  -88.16275218658868

The flux throughr EX_arg__L_e is:  0.0

The flux throughr MMO2 is:  88.16275218658868

The flux throughr ALCD1 is:  88.1627539405831

---Model 2---

Reaction R01142, solution of round 2 has been added to the model


In [23]:
consumption

{1: ({'model': 'Model1'},
  (['MMO2', 'ALCD1'],
   0.8769972144269785,
   {'MMO2': 88.16275218658868, 'ALCD1': 88.1627539405831},
   {'EX_ch4_e': -88.16275218658868},
   {'EX_arg__L_e': 0.0})),
 2: ({'model': 'Model2'},
  (['R01142', 'ALCD1'],
   0.8769972144269785,
   {'R01142': 110.201555639305, 'ALCD1': 110.20155739329942},
   {'EX_ch4_e': -110.201555639305},
   {'EX_arg__L_e': 0.0})),
 3: ({'model': 'Model3'},
  (['R01143', 'ALCD1'],
   0.8769972144269785,
   {'R01143': 132.2439718235817, 'ALCD1': 132.24397357757613},
   {'EX_ch4_e': -132.2439718235817},
   {'EX_arg__L_e': 0.0})),
 4: ({'model': 'Model4'},
  (['R01142', 'POX2', 'PRDX'],
   0.07906382561252327,
   {'R01142': 500.5788290740754,
    'POX2': 6.828205063204772,
    'PRDX': 500.578829232203},
   {'EX_ch4_e': -500.5788290740754},
   {'EX_arg__L_e': 0.0}))}

In [24]:
production = dict_prod_sol('./pipeline/inputs/ecoli_ch4_arg.csv', consumption, model_aux, universal)


Bounds of biomass during optimization of consumption =  (0.0, 1000.0)

Bounds of biomass during optimization of production =  (0.04384986072134892, 1000.0)

The metabs to produce are:  ['arg__L']

---1---
The model can already satisfy the objective

---2---
The model can already satisfy the objective

---3---
The model can already satisfy the objective

---4---
The model can already satisfy the objective


In [25]:
production

{'1': {'1': 9.082342543721138},
 '2': {'2': 8.844247835304447},
 '3': {'3': 8.685517676574253},
 '4': {'4': 8.010915181295902}}

In [27]:
final = cons_prod_dict('./pipeline/inputs/ecoli_ch4_arg.csv', model_aux, universal, consumption, production)

unknown metabolite 'qh2_c' created
Substrate's exchange reaction bounds :  (-6.6000000000000005, -5.4)
Target's exchange reaction bounds :  (2.0, 1000.0)
Carbon source:  []
Target:  EX_arg__L_e: arg__L_e --> 
FBA objective value:  2.0 
Substrate consumption flux:  -5.4 
Target production flux:  2.0 
Biomass:  0.0438498607213489 

pFBA is infeasible, control if the coefficients of the 
        reaction equation are correct (or use a different boudary 
        reaction of the target as model objective):  None (infeasible)
The thermodynamic analysis cannot proceed because of infeasible pFBA
The thermodynimac analysis has been unsuccesful
unknown metabolite 'qh2_c' created
Substrate's exchange reaction bounds :  (-6.6000000000000005, -5.4)
Target's exchange reaction bounds :  (2.0, 1000.0)
Carbon source:  []
Target:  EX_arg__L_e: arg__L_e --> 
FBA objective value:  2.0 
Substrate consumption flux:  -5.4 
Target production flux:  2.0 
Biomass:  0.0438498607213489 

pFBA is infeasible, contr

In [28]:
 final

{'consumption_1': ({'model': 'Model1'},
  (['MMO2', 'ALCD1'],
   0.8769972144269785,
   {'MMO2': 88.16275218658868, 'ALCD1': 88.1627539405831},
   {'EX_ch4_e': -88.16275218658868},
   {'EX_arg__L_e': 0.0})),
 'production_1': {'arg__L': {'EX_arg__L_e flux': 9.082342543721138,
   'thermodynamic': {'mdf': None, 'pathway_length': None}}},
 'consumption_2': ({'model': 'Model2'},
  (['R01142', 'ALCD1'],
   0.8769972144269785,
   {'R01142': 110.201555639305, 'ALCD1': 110.20155739329942},
   {'EX_ch4_e': -110.201555639305},
   {'EX_arg__L_e': 0.0})),
 'production_2': {'arg__L': {'EX_arg__L_e flux': 8.844247835304447,
   'thermodynamic': {'mdf': None, 'pathway_length': None}}},
 'consumption_3': ({'model': 'Model3'},
  (['R01143', 'ALCD1'],
   0.8769972144269785,
   {'R01143': 132.2439718235817, 'ALCD1': 132.24397357757613},
   {'EX_ch4_e': -132.2439718235817},
   {'EX_arg__L_e': 0.0})),
 'production_3': {'arg__L': {'EX_arg__L_e flux': 8.685517676574253,
   'thermodynamic': {'mdf': None, 'pathw

## Concluding considerations
This approach mixes some individual research of the candidate reactions to remove from the model to generate the auxotrophic strain before using the functions of the pipeline to find out if the auxotrophic model can grow on a particular substrate and produce a target. 


<h4 style="color: red;"> In principle both reaction addition and reaction removal for growth coupled production should be found, however, the module of the pipeline using Optknock has very long running times, thus the followin analysis is uncompleted </h4>

In [38]:
from pipeline.scripts.call_Optknock import full_knock_out_analysis

In [ ]:
ko_results = full_knock_out_analysis('./pipeline/inputs/ecoli_ch4_arg.csv', consumptionr, final, model_aux, universal)

Number of candidate reactions:  1914
---------1---------
Running OptKnock . . .
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 21661 rows, 15437 columns and 69247 nonzeros
Model fingerprint: 0xaba865cb
Variable types: 12725 continuous, 2712 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-06, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e-02, 1e+03]
  RHS range        [1e+00, 2e+03]
Presolve removed 11855 rows and 3898 columns
Presolve time: 0.61s
Presolved: 9806 rows, 11539 columns, 40847 nonzeros
Variable types: 9762 continuous, 1777 integer (1777 binary)

Root relaxation: objective 7.890598e+00, 3138 iterations, 0.57 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    7.89060    0    7          -    7.89060      -     -    1s
H    0     0                       0.0921881    7.89060  8459%     -    1

Minimal target rate  0.000:
Maximal target flux  0.092:
{1: {'KOs': [], 'Objective': 0.09218813182144459}, 2: {'KOs': ['MN2tpp'], 'Objective': 0.09788430681589072}}
Running OptKnock . . .
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 21661 rows, 15437 columns and 69247 nonzeros
Model fingerprint: 0x4830a635
Variable types: 12725 continuous, 2712 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-06, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e-02, 1e+03]
  RHS range        [1e+00, 2e+03]
Presolve removed 11982 rows and 3962 columns
Presolve time: 0.46s
Presolved: 9679 rows, 11475 columns, 40526 nonzeros
Variable types: 9727 continuous, 1748 integer (1748 binary)

Root relaxation: objective 7.890598e+00, 3083 iterations, 0.75 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    7.89060    0    5       

 40319 24359    7.85334   46   23    0.10887    7.85334  7113%  26.1  480s
 41092 24866    1.52884   76    8    0.10887    7.85231  7112%  26.0  485s
 41895 25092    0.22801  164    2    0.10887    7.82635  7089%  26.0  491s
 42081 25275    7.82635   47   27    0.10887    7.82635  7089%  26.2  496s
 42572 25732    2.89446  272    4    0.10887    7.82635  7089%  26.1  501s
 42802 25734    0.31497  117   11    0.10887    7.82635  7089%  26.0  508s
 42804 25792    0.31497  118   12    0.10887    7.82635  7089%  26.0  510s
 43094 26002    1.73125   88    4    0.10887    7.82635  7089%  26.1  515s
 43549 26011     cutoff   54         0.10887    7.82635  7089%  26.1  520s
 44076 26191    7.66646   42   21    0.10887    7.82635  7089%  26.0  525s
 44537 26493    7.82635   39   32    0.10887    7.82635  7089%  26.0  530s
 44849 26588     cutoff   53         0.10887    7.82635  7089%  26.1  535s
 45446 26869    7.75193   30    4    0.10887    7.82631  7089%  26.1  541s
 45724 27083    6.90212  

 81617 47583    1.74510   65   25    0.10887    7.26692  6575%  26.9 1046s
 82335 47891    6.12959   46   25    0.10887    7.26692  6575%  26.8 1052s
 82520 47972 infeasible   34         0.10887    7.26656  6574%  26.8 1055s
 83049 48151     cutoff   47         0.10887    7.26018  6569%  26.7 1060s
 83302 48408    2.98045  241    3    0.10887    7.26012  6568%  26.8 1066s
 83631 48659    2.87918  168   18    0.10887    7.26012  6568%  26.8 1072s
 83811 48658    7.26012   31    7    0.10887    7.26012  6568%  26.8 1075s
 84142 48712    7.26012   27    6    0.10887    7.26012  6568%  26.9 1082s
 84378 48888    1.60816   70   15    0.10887    7.26012  6568%  26.9 1085s
 84922 49205    5.95696   44   24    0.10887    7.26012  6568%  26.8 1091s
 85297 49351    2.28041   67   14    0.10887    7.26012  6568%  26.8 1097s
 85450 49420    3.69776   47   16    0.10887    7.26012  6568%  26.9 1100s
 85730 49641    2.84078   54   46    0.10887    7.26012  6568%  26.9 1106s
 86260 50010    7.25895  

 123782 68412    3.84038   60   25    0.10887    6.25623  5646%  28.4 1591s
 124249 68575     cutoff   55         0.10887    6.23466  5627%  28.4 1596s
 124585 68600 infeasible   71         0.10887    6.22534  5618%  28.4 1600s
 125265 69030    0.94555   97    4    0.10887    6.18735  5583%  28.4 1606s
 125821 69359     cutoff   75         0.10887    6.16108  5559%  28.4 1610s
 126389 69636    5.19838   52   21    0.10887    6.13054  5531%  28.3 1615s
 127285 70092     cutoff   52         0.10887    6.08542  5490%  28.2 1621s
 127761 70224    0.27241  134    5    0.10887    6.05305  5460%  28.1 1625s
 128590 70542 infeasible   34         0.10887    6.01417  5424%  28.0 1631s
 129060 70581    2.12167   67   20    0.10887    5.97699  5390%  28.0 1635s
 129309 70699    5.95711   41   14    0.10887    5.95882  5373%  28.1 1641s
 129685 70640 infeasible   47         0.10887    5.94118  5357%  28.2 1647s
 129802 70730    3.48909   64   27    0.10887    5.93394  5350%  28.3 1651s
 130612 7103

...

 667194 264869 infeasible  164         0.10887    2.12820  1855%  10.8 8273s
 667511 264793 infeasible  262         0.10887    2.12754  1854%  10.8 8283s
 667903 264730     cutoff  215         0.10887    2.12683  1854%  10.8 8289s
 668198 264543    0.24123  152    2    0.10887    2.12616  1853%  10.8 8293s
 668404 264100 infeasible  167         0.10887    2.12569  1852%  10.8 8308s
 668926 263968 infeasible   64         0.10887    2.12486  1852%  10.8 8315s
 669132 263928 infeasible  180         0.10887    2.12377  1851%  10.8 8320s
 669323 263689 infeasible  176         0.10887    2.12248  1850%  10.8 8328s
 669562 263560 infeasible  179         0.10887    2.12193  1849%  10.8 8339s
 669934 263401 infeasible  170         0.10887    2.12170  1849%  10.8 8344s
 670155 263277 infeasible  135         0.10887    2.12088  1848%  10.8 8349s
 670366 263282    0.37278  132   17    0.10887    2.12026  1847%  10.8 8364s
 670466 263418    1.08659  102   21    0.10887    2.12021  1847%  10.8 8373s

 701869 256754 infeasible  235         0.10887    2.04411  1778%  10.6 9119s
 702222 256620 infeasible  241         0.10887    2.04370  1777%  10.6 9122s
 702356 256514 infeasible  222         0.10887    2.04238  1776%  10.6 9125s
 702622 256219 infeasible  143         0.10887    2.03816  1772%  10.6 9132s
 702757 256172     cutoff  123         0.10887    2.03652  1771%  10.6 9137s
 702921 256085 infeasible  150         0.10887    2.03529  1769%  10.6 9143s


KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\delip\anaconda3\envs\cameo\lib\site-packages\ipykernel\iostream.py", line 387, in write
    if self.echo is not None:
KeyboardInterrupt


 703470 255597 infeasible  169         0.10887    2.03408  1768%  10.6 9157s
 703618 255582 infeasible  161         0.10887    2.03244  1767%  10.6 9162s
 703841 255442     cutoff  162         0.10887    2.03127  1766%  10.6 9166s
 703981 255371     cutoff   62         0.10887    2.02993  1765%  10.6 9170s
 704386 255600    1.03327   97    2    0.10887    2.02731  1762%  10.6 9181s


In [ ]:
ko_results